In [ ]:
!pip install --upgrade diffusers transformers accelerate torch bitsandbytes scipy safetensors xformers

In [ ]:
# ========================================
# COMPLETE COLAB SETUP - ULTRA HIGH QUALITY (SDXL)
# ========================================

# Install required packages
!pip install -q pyngrok flask diffusers transformers accelerate

from pyngrok import ngrok
from flask import Flask, request, send_file
import io
import torch
from diffusers import StableDiffusionXLPipeline

# ⚠️ REPLACE WITH YOUR NGROK TOKEN
# Get it from: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "Your-auth-token"

# Set auth token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

app = Flask(__name__)

# ========================================
# LOAD SDXL MODEL (BEST QUALITY)
# ========================================
print("="*60)
print("🎨 Loading SDXL Model (Ultra High Quality)...")
print("⚠️  This will take several minutes - please wait...")
print("="*60)

torch.cuda.empty_cache()

# Load SDXL for maximum quality
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)

pipe = pipe.to("cuda")
pipe.enable_attention_slicing()
pipe.enable_vae_slicing()

print("✅ SDXL Model loaded successfully!")
print("🚀 Ready for ULTRA HIGH-QUALITY generation!")
print("="*60)

# ========================================
# FLASK ENDPOINTS
# ========================================

@app.route('/generate', methods=['POST'])
def generate_image():
    """Generate ULTRA high-quality image from prompt"""
    try:
        prompt = request.form.get('prompt')

        if not prompt:
            return {'error': 'No prompt provided'}, 400

        print(f"\n{'='*60}")
        print(f"📝 Prompt: {prompt}")
        print(f"{'='*60}")
        print("🎨 Generating ULTRA high-quality image with SDXL...")

        # SDXL produces amazing quality with minimal prompt enhancement
        enhanced_prompt = f"{prompt}, masterpiece, best quality, highly detailed"

        # Generate with SDXL (native 1024x1024)
        with torch.no_grad():
            result = pipe(
                enhanced_prompt,
                negative_prompt="blurry, bad quality, worst quality, low resolution, jpeg artifacts, distorted, deformed",
                num_inference_steps=40,
                guidance_scale=7.5,
                generator=torch.Generator(device="cuda").manual_seed(42)
            )
            image = result.images[0]

        print("✅ Ultra high-quality image generated!")
        print("💾 Converting to PNG...")

        # Save with maximum quality
        buf = io.BytesIO()
        image.save(buf, format='PNG', optimize=False, compress_level=1)
        image_size = buf.tell()
        buf.seek(0)

        print(f"✅ Ready! Size: {image_size:,} bytes ({image_size/1024:.1f} KB)")
        print(f"📐 Resolution: 1024x1024 pixels")
        print(f"{'='*60}\n")

        # Return complete image
        image_data = buf.read()
        final_buf = io.BytesIO(image_data)

        return send_file(
            final_buf,
            mimetype='image/png',
            as_attachment=False,
            download_name='generated.png'
        )

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return {'error': str(e)}, 500

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return {
        'status': 'healthy',
        'model': 'stable-diffusion-xl-base-1.0',
        'resolution': '1024x1024',
        'quality': 'ultra'
    }

@app.route('/', methods=['GET'])
def home():
    """Home endpoint"""
    return '''
    <h1>🎨 AI Image Generator API (SDXL)</h1>
    <p>Model: Stable Diffusion XL 1.0</p>
    <p>Resolution: 1024x1024</p>
    <p>Quality: ULTRA HIGH</p>
    <p>Status: Online ✅</p>
    <p>POST to /generate with 'prompt' parameter</p>
    '''

# ========================================
# START SERVER
# ========================================

if __name__ == "__main__":
    try:
        # Start ngrok tunnel
        print("\n" + "="*60)
        print("🚀 Starting ngrok tunnel...")
        print("="*60)

        public_url = ngrok.connect(5000)

        print("\n" + "🎉"*30)
        print("\n✅ SDXL SERVER SUCCESSFULLY STARTED!\n")
        print("🎉"*30)
        print("\n" + "="*60)
        print("📡 NGROK PUBLIC URL:")
        print("="*60)
        print(f"\n    {public_url}\n")
        print("="*60)
        print("\n⚠️  IMPORTANT - COPY THIS URL!")
        print("="*60)
        print(f"\n1. Copy this URL: {public_url}")
        print(f"\n2. Open your script.py file")
        print(f"\n3. Update the COLAB_URL line to:")
        print(f"\n   COLAB_URL = '{public_url}/generate'")
        print(f"\n4. Save and run: python script.py")
        print(f"\n5. Open browser: http://localhost:5000")
        print("\n" + "="*60)
        print("\n📊 Server Info:")
        print("   - Model: SDXL 1.0 (Best Available)")
        print("   - Resolution: 1024x1024 pixels")
        print("   - Quality: ULTRA HIGH (Professional Grade)")
        print("   - Time: ~6-8 minutes per image")
        print("\n" + "="*60 + "\n")

        # Run Flask app
        app.run(port=5000, host='0.0.0.0')

    except Exception as e:
        print(f"\n❌ Error starting server: {str(e)}")
        print("\n⚠️  If you see 'authentication failed', you need to:")
        print("1. Go to: https://dashboard.ngrok.com/get-started/your-authtoken")
        print("2. Copy your authtoken")
        print("3. Replace NGROK_AUTH_TOKEN in the code above")